In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_csv('./mfdexpt.csv',index_col=0) # Specify directory

# Get base sensor names (exclude BG columns)
sensors = sorted({
    re.split(r'\.', col)[0]
    for col in data.columns
    if not col.endswith('BG')
})

for sensor in sensors:
    bg_col = f"{sensor}BG"
    
    # Skip if BG column doesn't exist
    if bg_col not in data.columns:
        print(f"Warning: {bg_col} not found — skipping {sensor}")
        continue
    
    # Find all replicate columns for this sensor
    sensor_cols = [
        col for col in data.columns
        if re.split(r'\.', col)[0] == sensor
    ]
    
    # Subtract background
    data[sensor_cols] = data[sensor_cols].sub(data[bg_col], axis=0)

# Get rid of BG columns 
data = data.loc[:, ~data.columns.str.endswith('BG')]

# Get rid of bad frames
df_filled = data.copy()
for row in range(1, df_filled.shape[0]-1):  # avoid first and last row to prevent index error
    for col in df_filled.columns:
        if df_filled.at[row, col] == 0:
            df_filled.at[row, col] = (df_filled.at[row-1, col] + df_filled.at[row+1, col]) / 2

# Subtract mean of promoterless
data = df_filled
promoterless_cols = [c for c in data.columns if c.startswith('U139') and 'BG' not in c]
promoterless_mean = data[promoterless_cols].mean(axis=1)
data = data.sub(promoterless_mean, axis=0)
data = data+50

# Signal to noise, noise defined as STD of preinduction signal
SNR = data/data.iloc[0:45, :].std(axis=0)

sensor_names = [col.split('.')[0] for col in SNR.columns]

# Create an empty dictionary to hold the results
summary = {}

for sensor in set(sensor_names):
    # Select columns belonging to this sensor
    cols = [col for col in SNR.columns if col.startswith(sensor)]
    
    # Compute mean and std across these columns (axis=1 → row-wise)
    summary[f'{sensor}_mean'] = SNR[cols].mean(axis=1)
    summary[f'{sensor}_std'] = SNR[cols].std(axis=1)

# Convert dictionary to a DataFrame
summary_df = pd.DataFrame(summary)

# UNCOMMENT TO SAVE AGAIN 
# summary_df.to_csv('SNR.csv')